In this tutorial, we will take a brief look at the internals of ```CrypTensors```. 

This tutorial is optional, and can be skipped without any loss of continuity to the following tutorials.

In [1]:
#import the libraries
import crypten
import torch

#initialize crypten
crypten.init()

A `CrypTensor` is a tensor encrypted using secure MPC protocols, called `MPCTensor`, since we use secure MPC as the cryptographic protocol in CrypTen currently. (We plan to support additional cryptographic protocols for privacy-preserving machine learning in the future, and when we do so, a `CrypTensor` will be a tensor encrypted with any of those protocols.)

In order to support the mathematical operations required by the `MPCTensor`, CrypTen implements two kinds of secret-sharing protocols: arithmetic (or additive) secret-sharing and binary (or XOR) secret-sharing. Arithmetic secret sharing forms the basis of most of the mathematical operations implemented by `MPCTensor`. On the other hand, binary secret-sharing allows greater efficiency in evaluating logical expressions. 

`MPCTensors` encrypted with arithmetic secret-sharing are internally `ArithmeticSharedTensors`. `MPCTensors` encrypted with binary secret-sharing are internally `BinarySharedTensors`. 

In this tutorial, we'll first introduce the concept of a `CrypTensor` `ptype` (for <i>private-type</i>), and show how to use it to obtain `ArithmeticSharedTensors` and `BinarySharedTensors`. We'll then look more closely at operations supported by `ArithmeticSharedTensor` and `BinarySharedTensor`. Finally, we'll describe how CrypTen handles an operation that needs to use both types of secret-sharing for its tensors.


### <i>ptype</i> in CrypTen
CrypTen defines the `ptype` (for <i>private-type</i>) attribute of a `CrypTensor` to denote the kind of secret-sharing protocol used in the `CrypTensor`. The `ptype` is, in many ways, analogous to the `dtype` of PyTorch. The `ptype` may have two values: 
<ul>
<li>`crypten.arithmetic` for `ArithmeticSharedTensors`</li>
<li>`crypten.binary` for  `BinarySharedTensors`</li>
</ul>
We can use the `ptype` attribute to create a `CrypTensor` with the appropriate secret-sharing protocol. For example: 

In [2]:
#Constructing CrypTensors with ptype attribute

#arithmetic secret-shared tensors
x_enc = crypten.cryptensor([1.0, 2.0, 3.0], ptype=crypten.arithmetic)
print("x_enc internal type:", x_enc.ptype)

#binary secret-shared tensors
y = torch.tensor([1, 2, 1], dtype=torch.int32)
y_enc = crypten.cryptensor(y, ptype=crypten.binary)
print("y_enc internal type:", y_enc.ptype)

x_enc internal type: ptype.arithmetic
y_enc internal type: ptype.binary


### ArithmeticSharedTensor
Let's now look more closely at the ```ArithmeticSharedTensor```. The ```ArithmeticSharedTensor``` implements arithmetic secret sharing, which forms the basis of most of the mathematical operations implemented by ```MPCTensor```. The creation of an ```ArithmeticSharedTensor``` and its use in arithmetic operations is similar to what we've seen in Tutorial 1. There is, however, one difference to note: division of an ```ArithmeticSharedTensor``` requires the divisor to be a plaintext tensor, due to the range of operations supported by an ```ArithmeticSharedTensor```.

In [3]:
from crypten.mpc.primitives import ArithmeticSharedTensor 

#creating an ArithmeticSharedTensor
x_enc = ArithmeticSharedTensor([1, 2, 3])

#operations with plaintext tensors
y_enc = x_enc + 2
print("addition:", y_enc.get_plain_text())

y_enc = x_enc - 2
print("subtraction:", y_enc.get_plain_text())

y_enc = x_enc * 2
print("multiplication:", y_enc.get_plain_text())

y_enc = x_enc / 2
print("division: ", y_enc.get_plain_text())

addition: tensor([3., 4., 5.])
subtraction: tensor([-1.,  0.,  1.])
multiplication: tensor([2., 4., 6.])
division:  tensor([0.5000, 1.0000, 1.5000])


In [4]:
#operations with encrypted tensors
y_enc = ArithmeticSharedTensor([2])

z_enc = x_enc + y_enc
print("addition:", z_enc.get_plain_text())

z_enc = x_enc - y_enc
print("subtraction:", z_enc.get_plain_text())

z_enc = x_enc * y_enc
print("multiplication:", z_enc.get_plain_text())

addition: tensor([3., 4., 5.])
subtraction: tensor([-1.,  0.,  1.])
multiplication: tensor([2., 4., 6.])


How do we know the `ArithmeticSharedTensor` does not directly contain our plaintext tensor? We'll look inside the tensor held by an `ArithmeticSharedTensor` to see this, but first, we need to understand a little more about how encryption works in CrypTen.

CrypTen follows the standard MPI programming model: it runs a separate process for each party, but each process runs the identical (complete) program. Each process has a `rank` variable to identify itself. If a process rank `i` owns (and loads) the plaintext data `x`, then `x` gets encrypted with `i` as its source value (denoted as `src`). (This allows process rank `i` will generate the secret shares of `x`, and communicate them to all the other processes.)

In the next example, we'll show how to use the `rank` and `src` values to encrypt tensors. After we encrypt each tensor, we'll use the attribute `.share` to access the data in the encrypted tensor.

<i>(Technical note: Since Jupyter notebooks only run a single process, we use a custom decorator `mpc.run_multiprocess` to simulate a multi-party world in our notebooks.)</i>

In [5]:
import crypten.mpc as mpc
import crypten.communicator as comm #Crypten communicator, used to access rank
 
@mpc.run_multiprocess(world_size=2)
def examine_arithmetic_shares():
    z = torch.tensor([1, 2, 3])
    
    rank = comm.get().get_rank()
    print(f"Rank {rank}: Plaintext:", z)
    
    #encrypting the plaintext tensor z:    
    #for rank 0 process:
    x_enc = ArithmeticSharedTensor(z, src=0)
    #for rank 1 process: 
    y_enc = ArithmeticSharedTensor(z, src=1)
    
    #access the tensor inside the ArithmeticSharedTensor
    print(f"Rank {rank}: x_enc share:", x_enc.share)
    print(f"Rank {rank}: y_enc share:", y_enc.share)
        
x = examine_arithmetic_shares()

Rank 0: Plaintext: tensor([1, 2, 3])
Rank 1: Plaintext: tensor([1, 2, 3])
Rank 1: x_enc share: tensor([6507446633756756391, 8933045155855769194, 4941929691160179679])
Rank 0: x_enc share: tensor([-6507446633756690855, -8933045155855638122, -4941929691159983071])
Rank 1: y_enc share: tensor([  422154116776664286,  6094858192667170110, -8250182259831619938])
Rank 0: y_enc share: tensor([ -422154116776598750, -6094858192667039038,  8250182259831816546])


Note that the two shares of `x_enc`  are not the plaintext tensor `z`, and neither are the shares of `y_enc`. Further, note the shares created for `z` are different in `x_enc` and in `y_enc`. This is because we used `src=0` to create `x_enc`, and `src=1` to create `y_enc`. So, process 0 creates the secret shares for `z` in `x_enc` using its (secret) random seed, and provides process 1 its share of `x_enc`. Likewise, in the next step, process 1 (separately) creates the secret shares for `z` in `y_enc` using its (different, secret) random seed, and provides process 0 its share of `y_enc`. 

This concludes our overview of ```ArithmeticSharedTensors```. Next, we'll look at ```BinarySharedTensors```.
<br>
### BinarySharedTensors
The second type of secret-sharing implemented in CrypTen is binary or XOR secret-sharing. This type of secret-sharing allows greater efficiency in evaluating logical expressions. 

We can create a ```BinarySharedTensor``` just as we did an ```ArithmeticSharedTensor```. However, the ```BinarySharedTensor``` supports binary logical operations, rather than arithmetic operations. Thus, for example: 

In [6]:
from crypten.mpc.primitives import BinarySharedTensor

#creating a BinarySharedTensor. 
x = torch.tensor([1, 2, 3])
x_enc = BinarySharedTensor(x)

#operations between BinarySharedTensor/plaintext tensors
y_enc = x_enc ^ 3
print("XOR operation:", y_enc.get_plain_text())

y_enc = x_enc & 3
print("AND operation:", y_enc.get_plain_text())

y_enc = x_enc | 3
print("OR operation:", y_enc.get_plain_text())

XOR operation: tensor([2, 1, 0])
AND operation: tensor([1, 2, 3])
OR operation: tensor([3, 3, 3])


In [7]:
#creating a second BinarySharedTensor
y = torch.tensor([4, 1, 3], dtype=torch.int32)
y_enc = BinarySharedTensor(y)

#operations between two BinarySharedTensors
z_enc = x_enc ^ y_enc
print("XOR operation:", z_enc.get_plain_text())

z_enc = x_enc & y_enc
print("AND operation:", z_enc.get_plain_text())

z_enc = x_enc | y_enc
print("OR operation:", z_enc.get_plain_text())

XOR operation: tensor([5, 3, 0])
AND operation: tensor([0, 0, 3])
OR operation: tensor([5, 3, 3])


As we did with the `ArithmeticSharedTensor`, we can look into the `BinarySharedTensor` to see that it does not actually keep the plaintext tensor. We can access the tensor held by the `BinarySharedTensor` with the `.share` attribute. As we saw with the `ArithmeticSharedTensors`, in a multi-party world, each process will need to encrypt its data with its own `rank`. Notice that once again, the shares in `x_enc` and `y_enc` are distinct from each other.

In [8]:
from crypten.mpc.primitives import BinarySharedTensor

@mpc.run_multiprocess(world_size=2)
def examine_binary_tensor():
    
    z = torch.tensor([1, 2, 3], dtype=torch.int32)
    
    rank = comm.get().get_rank()
    print(f"Rank {rank}: Plaintext:", z)

    #Encrypt as BinarySharedTensor:
    #for rank 0 process
    x_enc = BinarySharedTensor(z, src=0)   
    #for rank 1 process
    y_enc = BinarySharedTensor(z, src=1)

    #access the tensor inside the BinarySharedTensor
    print(f"Rank {rank}: x_enc share:", x_enc.share)
    print(f"Rank {rank}: y_enc share:", y_enc.share)
    
x = examine_binary_tensor()

Rank 1: Plaintext: tensor([1, 2, 3], dtype=torch.int32)
Rank 0: Plaintext: tensor([1, 2, 3], dtype=torch.int32)
Rank 0: x_enc share: tensor([-5329831728445400837,  3124917732480312484, -5598881425843780996])
Rank 1: x_enc share: tensor([-5329831728445400838,  3124917732480312486, -5598881425843780993])
Rank 0: y_enc share: tensor([ -505704474185003628, -8732970886542952507, -3492699824048994491])
Rank 1: y_enc share: tensor([ -505704474185003627, -8732970886542952505, -3492699824048994490])


### Using Both Secret-sharing Protocols
Quite often a mathematical function may need to use both additive and XOR secret sharing for efficient evaluation. CrypTen provides functionality that allows for the conversion of `ArithmeticSharedTensor` to `BinarySharedTensor` and vice-versa.

In [9]:
from crypten.mpc import MPCTensor

@mpc.run_multiprocess(world_size=2)
def examine_conversion():
  
    #plaintext tensor
    x = torch.tensor([1, 2, 3])
    
    rank = comm.get().get_rank()
    
    print(f"Rank {rank}: Plaintext:", x)
    print()

    #create an MPCTensor with arithmetic secret sharing
    x_enc_arithmetic = MPCTensor(x, ptype=crypten.arithmetic, src=0)
    print(f"Rank {rank}: Initial encrypted type:", x_enc_arithmetic.ptype)
    print(f"Rank {rank}: ArithmeticSharedTensor value:", x_enc_arithmetic.share)
    print()

    #convert the MPCTensor with binary secret sharing
    x_enc_binary = x_enc_arithmetic.binary()
    print(f"Rank {rank}: After conversion:", x_enc_binary.ptype)
    print(f"Rank {rank}: BinarySharedTensor value:", x_enc_binary.share)
    print()

    #Decrypting the converted tensor yields the original tensor
    x_decrypted = x_enc_binary.get_plain_text()
    print(f"Rank {rank}: Decrypted:", x_decrypted)
        
z = examine_conversion()

Rank 0: Plaintext: tensor([1, 2, 3])
Rank 1: Plaintext: tensor([1, 2, 3])


Rank 0: Initial encrypted type: ptype.arithmetic
Rank 1: Initial encrypted type: ptype.arithmetic
Rank 0: ArithmeticSharedTensor value: tensor([ 7894822374111859623,  8363996642151064510, -2167477811202486943])
Rank 1: ArithmeticSharedTensor value: tensor([-7894822374111794087, -8363996642150933438,  2167477811202683551])


Rank 1: After conversion: ptype.binary
Rank 0: After conversion: ptype.binary
Rank 1: BinarySharedTensor value: tensor([ 8912641549483668076,  1784415293112309463, -4818247685583860145])
Rank 0: BinarySharedTensor value: tensor([ 8912641549483733612,  1784415293112178391, -4818247685583794609])


Rank 1: Decrypted: tensor([1., 2., 3.])
Rank 0: Decrypted: tensor([1., 2., 3.])


Thus, even with conversions between `ArithmeticSharedTensors` and `BinarySharedTensors`, CrypTen correctly decrypts the tensor.

For the algorithms that convert between `ArithmeticSharedTensors` and `BinarySharedTensors`, as well as examples of mathematical functions that require such conversion, please see the CrypTen documentation.